In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import pickle

In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

#############################################################################################
stop_words.extend(['from', 'also', 'would', 'say', 'percent', 'year', 'mention', 'make_sure', 'enough',
                   'yet', 'next','instead','perhaps', 'purely','thompson', 'matthew', 'charle', 'october', 'onetime',  'olivia', 'come', 'mike',  'martin',  'poorly', 
                   'rainy_day',  'way', 'many', 'often', 'make', 'give',  'use',   'may','great', 'well',
                   'take', 'see', 'provide', 'time', 'example', 'good', 'feel', 'large', 'mark', 'include', 'sai', 'lalao', 'coffee', 'usaid',
                   'become', 'get', 'high', 'stay', 'day', 'cut', 'increase', 'even', 'number', 'organization', 'process', 'result', 'affiliate', 
                   'offer', 'ict', 'white', 'jackson','susan','go','first','second'])

print(stop_words)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
# Import Dataset
article = pd.read_csv('analysis.csv')
df = pd.DataFrame(data=article['text'])

# Convert to list
data = df.values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=30) # higher threshold fewer phrases.  #=100
trigram = gensim.models.Phrases(bigram[data_words], threshold=30)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
def countword(data):
    l = len(data)
    number = 0
    for i in range(l):
        number = number + len(data[i])
    return(number)
        

In [9]:
# Remove Stop Words# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#Filter stop words again
data_lemmatized = remove_stopwords(data_lemmatized)

#print(data_lemmatized[:1])

233
233
233
233


In [16]:
print(countword(data_words))
print(countword(data_words_nostops))
print(countword(data_words_bigrams))
print(countword(data_lemmatized))

979882
557356
528221
477821


In [7]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [8]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ability', 2),
  ('able', 2),
  ('absolute', 1),
  ('accept', 1),
  ('access', 1),
  ('accommodate', 1),
  ('accomplish', 2),
  ('accomplished', 1),
  ('account', 1),
  ('achieve', 6),
  ('achievement', 1),
  ('across_sector', 1),
  ('act', 1),
  ('acting', 1),
  ('action', 1),
  ('actively', 1),
  ('activity', 1),
  ('actor', 1),
  ('actual', 1),
  ('adapt', 1),
  ('adaptation', 1),
  ('adaptive', 2),
  ('additional', 1),
  ('address', 1),
  ('adhere', 1),
  ('adherence', 1),
  ('adirondack', 1),
  ('adjustment', 1),
  ('administration', 2),
  ('admirable', 1),
  ('adulthood', 1),
  ('advantage', 1),
  ('advocacy', 3),
  ('agency', 5),
  ('agenda', 1),
  ('agree', 1),
  ('agreed_upon', 1),
  ('agreement', 3),
  ('aid_agencie', 1),
  ('aim', 2),
  ('airport', 1),
  ('align', 1),
  ('allocate', 1),
  ('allow', 3),
  ('almost', 2),
  ('already', 1),
  ('amazon', 6),
  ('amazon_region', 6),
  ('ambitious', 4),
  ('amendment', 1),
  ('amount', 2),
  ('ancestral', 1),
  ('andrew', 2),
  

In [9]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10,  
                                            random_state=70,
                                            #update_every=1,
                                            chunksize=160,
                                            eta='auto',
                                            iterations=1350,
                                            passes=12,
                                            eval_every=1,
                                            alpha='auto', 
                                            per_word_topics=True
                                           )

## Save LDA model and other data

In [1]:
#save LDA model data
print(lda_model)
lda_model.save('lda.model')
# later on, load trained model from file
lda_model =  gensim.models.LdaModel.load('lda.model')

NameError: name 'lda_model' is not defined

In [29]:
with open("savecorpus.pkl", "wb") as f:
    pickle.dump(corpus, f)

with open("data_lemmatized.pkl", "wb") as f:
    pickle.dump(data_lemmatized, f)
    
with open("id2word.pkl", "wb") as f:
    pickle.dump(id2word, f)